In [1]:

# -*- coding: utf-8 -*-
"""
Készíts osztályok segítségével egy nyilvántartást egy könyvesbolt számára:
A könyvek adatai:  cím, szerző, ár, készlet darabszám, selejtre kijelölve (True/False)
Készíts metódusokat (vagy ahol célszerűbb, függvényeket) :
1) txt fájlból való adat feltöltésre és fájlba történő mentésre (fájlnév mint paraméter) 
    Kész
2) 5-nél kevesebb készletű könyvek listázására  
    Kész
    
3) Keresés cím vagy szerző töredékek szerint nincs 
    Kész
    
4) Selejtezésre: a selejtezésre kijelölt  könyveket ki kell íratni egy selejt.txt fájlba és eltávolítani az "adatbázisból"
    Kész
5) Eladásra: Adott könyv készlet csökkentése megadott darabszámmal
    
    
6) A teljes készlet értékének kiszámítására
    Kész
"""

import pandas as pd
#ha egy osztálynak van külső module függősége, azt hogyan érdemes kezelni ?

import io
#io modul nélkül nem tudtam utf8-ba menteni



class KonyvNyilvantarto:
    #egy lista, ami futásidő alatt tartalmazza az összes könyvet
    Konyvek = []
    #filebeolvasás után lementjük a file elérését, hogy bárhol tudjunk rá hivatkozni
    KonyvekDB = ''
    
    def __init__(self,cim,szerzo,ar,keszlet_darabszam,selejt = False):
        self.cim = cim
        self.szerzo = szerzo
        self.ar = ar
        self.keszlet_darabszam = keszlet_darabszam
        self.selejt = selejt
        
       
      
    def bookReader(filepath):
        """
        Beolvassa  a beolvasni kívánt txt-t
        Bemeneti paraméter a file elérése
        """
        KonyvNyilvantarto.KonyvekDB = filepath
        def bookEntityMaker(entity):
            #nem futott le, ha a class hatókörébe definiáltam, ne felejtsek el rákérdezni Jánosnál
            for index, row in entity.iterrows():
                """
                Végig iterálunk az összes soron
                """
                #tesztelési célra kiíratás
                #print(row['cim'], row['szerzo'], row['ar'], row['keszlet_darabszam'], row['selejt'])
                "A klasszban szereplő listába bele appendelünk egy osztálypéldányt a txt fileból"
                KonyvNyilvantarto.Konyvek.append(KonyvNyilvantarto(row['cim'], row['szerzo'], row['ar'], row['keszlet_darabszam'], row['selejt']))
                
             #bookEntityMaker definíció vége  
        df = pd.read_csv(filepath,delimiter=',')
        #Az üres selejt oszlopok esetén felveszek egy alapértelmezett False értéket
        df['selejt'] = df['selejt'].fillna(value = 'False')
        #meghívom a bookEntityMakert, hogy a Konyvek class  listába a txt minden egyes sorát objektumként belementem
        bookEntityMaker(df)
        #a return értéke valójában teljesen felesleges, viszont előfordulhat, hogy hasznos, ha megkapjuk a beolvasott értékeket DF-ként, hogy a Pandas metódusait használjuk.
        return df
    
    def alacsonyKeszlet():
            """
            Konyvek listán iterálunk végig és megnézzük melyek azok a könyvek, melyekből készlet hiány van.
            Készlethiány == 5-nél kevesebb
            """
            for  sor in KonyvNyilvantarto.Konyvek:
                if sor.keszlet_darabszam < 5:
                    print(sor.cim)
                    
    def filmVSzerzoKeres(keresettString):
        """ 
        Bemeneti paraméter a keresett szöveg.
        A függvény célja, hogy keressen a címekben és a szerzőkben és visszaadja azokat a könyveket, ahol egyezőséget talál
        """
        for sor in KonyvNyilvantarto.Konyvek:
            if keresettString in sor.cim or keresettString in sor.szerzo:
                print(sor.cim + ' '+ sor.szerzo)
                    
                    
    def selejtezes():
        """
        A függvény kigyüjti a selejtezésre jelölt könyveket kiírja egy fileba, majd újra írja az eredeti file-t selejtezett könyvek nélkül
        Bementi paraméter az a file, ami az adatbázis alapjául szolgál
        
        """
        #ideiglenes lista csak azért, hogy kilehessen iratni a selejteket
        selejtek = []
        #feltöltöm a selejtek listát, illetve kiveszem az eredeti listából a selejtezett könyveket
        for  sor in KonyvNyilvantarto.Konyvek:
            if sor.selejt == True:
                selejtek.append(sor)
                KonyvNyilvantarto.Konyvek.remove(sor)
                
                
        #selejt txt feltöltése
        with io.open('selejt.txt', 'w',encoding='utf8') as selejtFajl:
            for sor in selejtek:
                selejtFajl.write(sor.cim + ','  + sor.szerzo + ',' + str(sor.ar) + ',' + str(sor.keszlet_darabszam) + ',' + str(sor.selejt) + '\n' ) 
                
              
                
        #adatbázis file újraíratása      
        with io.open(KonyvNyilvantarto.KonyvekDB, 'w',encoding='utf8') as nemSelejtFajl:
            #note to self ha legközelebb agyat növesztek, akkor az egészfejlécet lementem egy változóba, hogy egyhelyen kezeljem.
            nemSelejtFajl.write('cim,szerzo,ar,keszlet_darabszam,selejt\n' )
            for sor in KonyvNyilvantarto.Konyvek:
                nemSelejtFajl.write(sor.cim + ','  + sor.szerzo + ',' + str(sor.ar) + ',' + str(sor.keszlet_darabszam) + ',' + str(sor.selejt) + '\n' ) 
                
    def rendeles():
        """
        A függvény egy konkrét rendelést valósít meg, ahol a bemenet
        Kilistázza a könyvet és, hogy hány DB van belőle
        """
        #érdemes majd tovább fejleszteni azzal, hogy a fileban is csönketse a készlet darabszámot ne csak a listában
        print('-------------------------------------------------')
        print('Válasz a rendelni kívánt könyvekből (első számjegy a könyv azonosítója, illetve, hogy hány darabot szeretnél rendelni belőle')
        for sor in KonyvNyilvantarto.Konyvek:
            print( str(KonyvNyilvantarto.Konyvek.index(sor)+ 1) +' ' +sor.cim + ','  + sor.szerzo + ',' + str(sor.ar) + ',' + str(sor.keszlet_darabszam) + ',' + str(sor.selejt) + '\n' ) 
        KonyvAzon = -1
        #kiválasztunk egy könyvet
        while(len(KonyvNyilvantarto.Konyvek) > KonyvAzon and KonyvAzon <= -1):
            KonyvAzon = int(input('Rendelni kívánt könyv Azonosítója:',))
        
        KonyvAzon = KonyvAzon-1
        keszlet = KonyvNyilvantarto.Konyvek[KonyvAzon].keszlet_darabszam
        print('A Kiválasztott Könyv ' +KonyvNyilvantarto.Konyvek[KonyvAzon].cim +' amelyből jelenleg ' + str(keszlet) + ' DB van készleten')
        KonyvDarab = keszlet + 1
        print('Hány darabot szeretnél rendelni a könyvből ?')
        #kiválasztjuk, hogy hány darabot szeretnénk
        while (KonyvDarab > keszlet):
            KonyvDarab = int(input('Kérlek add meg a mennyiséget, amennyit szeretnél rendelni: ',))
            if KonyvDarab > keszlet:
                print('Túl sokat szeretnél')
        #csökkentjük a készletet majd kiíratjuk mennyi maradt
        KonyvNyilvantarto.Konyvek[KonyvAzon].keszlet_darabszam = KonyvNyilvantarto.Konyvek[KonyvAzon].keszlet_darabszam - KonyvDarab 
        print('Már csak '+  str(KonyvNyilvantarto.Konyvek[KonyvAzon].keszlet_darabszam) + 'DB maradt')
        
        

        
    def raktarErtek():
        """
        A teljes raktárkészlet értékét adja vissza, illetve iratja ki
        """
        raktarErtekAccum = 0           
        for  sor in KonyvNyilvantarto.Konyvek:
            raktarErtekAccum = raktarErtekAccum + (sor.ar * sor.keszlet_darabszam)
        
        return raktarErtekAccum
        
                
        
                

                
        
     

        

        
        

"""
#Fő program kezdete
"""

"""
#Teszt adatok
K1 = KonyvNyilvantarto('Baljós Árnyak','Terry Brooks',3000,10)
K2 = KonyvNyilvantarto('Klónok Támadása','R.A. Salvatore',5000,5)
K3 = KonyvNyilvantarto('A sithek bosszúja','Matthew Stover',500,40)


KonyvNyilvantarto.Konyvek.append(K1)
KonyvNyilvantarto.Konyvek.append(K2)
KonyvNyilvantarto.Konyvek.append(K3)
"""

1.
KonyvNyilvantarto.bookReader('books.txt')

2.
KonyvNyilvantarto.alacsonyKeszlet()

3.
KonyvNyilvantarto.filmVSzerzoKeres('Baljós')

4

KonyvNyilvantarto.selejtezes()
5.
KonyvNyilvantarto.rendeles()

6.
print(KonyvNyilvantarto.raktarErtek())




FileNotFoundError: [Errno 2] File books.txt does not exist: 'books.txt'